## My Trainer

In [ ]:
%cd /home/q123/Desktop/explo/

from mushroom_rl.environments.dm_control_env import DMControl
from mushroom_rl.policy import DeterministicPolicy
from src.ddpg import DDPG
from src.helpers import setup_experiment
from src.config import get_configs
import torch
from src.approximators.actor import ActorNetwork
from src.approximators.critic import CriticNetwork


# MDP
horizon = 500
gamma = 0.99
gamma_eval = 1.
#mdp = DMControl('walker', 'stand', horizon, gamma)


# Settings
initial_replay_size = 500
max_replay_size = 10000
batch_size = 200
n_features = 80
tau = .001


from os import path
import logging
log_file_path = path.join("/home/q123/Desktop/explo/logging.conf")
logging.config.fileConfig(log_file_path)

log_file_path

print("hello")

: 

In [2]:
def critic_step(agent,transitions,n_epochs=2):
    
    agent._replay_memory.add(transitions)

    state, action, reward, next_state, absorbing, _ =\
        agent._replay_memory.get(agent._batch_size())

    q_next = agent._next_q(next_state, absorbing)
    q_target = reward + agent.mdp_info.gamma * q_next

    agent._critic_approximator.fit(state, action, q_target,n_epochs=n_epochs,
                                    **agent._critic_fit_params)
    
    agent._update_target(agent._critic_approximator,
                        agent._target_critic_approximator)

In [9]:
%cd /home/q123/Desktop/explo/src

from src.optimizers.esq_pytorch import ESQOptimizer


env_name = "Swimmer-v4"
kernel_name = "rbf"


if __name__ == '__main__':
    
    ## Setup environment
    env_config,likelihood_config,kernel_config,optimizer_config,trainer_config = get_configs(env_name,kernel_name)
    _,env = setup_experiment(env_config,kernel_config,likelihood_config,additional_layers=[])


    # Setup DDPG
    mdp = env.env

    policy_class = DeterministicPolicy
    policy_params = dict()

    actor_input_shape = mdp.info.observation_space.shape
    actor_params = dict(network=ActorNetwork,
                        n_features=n_features,
                        input_shape=actor_input_shape,
                        output_shape=mdp.info.action_space.shape)

    actor_optimizer = {'class': torch.optim.Adam,
                    'params': {'lr': 1e-4}}

    critic_input_shape = (actor_input_shape[0] + mdp.info.action_space.shape[0],)
    critic_params = dict(network=CriticNetwork,
                        optimizer={'class': torch.optim.Adam,
                                    'params': {'lr': 1e-3}},
                        loss=F.mse_loss,
                        n_features=n_features,
                        input_shape=critic_input_shape,
                        output_shape=(1,))

    agent = DDPG(mdp.info, policy_class,policy_params,
                actor_params, actor_optimizer, 
                critic_params,
                batch_size, initial_replay_size, max_replay_size,
                tau)

    esq_optimizer = ESQOptimizer(
                            critic=agent._critic_approximator.model.network,
                            actor = agent._actor_approximator.model.network,
                            sigma=1e-1,
                            params_per_step=40,
                            n_workers=8)
 
    for i in range(100):
        
        avg_reward,states,transitions = env.run_many(agent._actor_approximator,5)
        print("avg_reward",avg_reward)  
            
        critic_step(agent,transitions)
        
        states_batch,_,_,_,_,_ = agent._replay_memory.get(agent._batch_size())
        
        print("done training critic")
        grads = esq_optimizer.step(torch.tensor(states_batch)) ## fit critic
        #grads = esq_optimizer.step(states) ## fit critic
        
        #print(agent._critic_approximator.model.network.parameters())
        
    #     #print(optimizer.actor.parameters())

/home/q123/Desktop/explo/src
MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]
Using ard_num_dims = 18
avg_reward tensor([-0.0531])
done training critic
avg_reward tensor([-0.0628])
done training critic
avg_reward tensor([-0.0645])
done training critic
avg_reward tensor([-0.0505])
done training critic
avg_reward tensor([-0.0483])
done training critic
avg_reward tensor([-0.0462])
done training critic
avg_reward tensor([-0.0634])
done training critic
avg_reward tensor([-0.0520])
done training critic
avg_reward tensor([-0.0425])
done training critic
avg_reward tensor([-0.0493])
done training critic
avg_reward tensor([-0.0596])
done training critic
avg_reward tensor([-0.0609])
done training critic
avg_reward tensor([-0.0612])
done training critic
avg_reward tensor([-0.0585])
done training critic
avg_reward tensor([-0.0515])
done training critic
avg_reward tensor([-0.0506])
done training critic
avg_reward tensor([-0.0605])
done training critic
avg_reward tensor([-0.0514])
done trainin

In [ ]:
# from mushroom_rl.approximators import Regressor
# from mushroom_rl.approximators.parametric import TorchApproximator
# from mushroom_rl.utils.replay_memory import ReplayMemory

# replay_memory = ReplayMemory(initial_replay_size, max_replay_size)
# critic_approximator = Regressor(TorchApproximator,
#                                               **critic_params)
